In [1]:
import pyodbc
import pandas as pd
import numpy as np

### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')

### Get the students table
students = pd.read_sql("SELECT * FROM Students;", conn)
courses = pd.read_sql("SELECT * FROM Courses;", conn)
classrooms = pd.read_sql("SELECT * FROM Classrooms;", conn)
teachers = pd.read_sql("SELECT * FROM Teachers;", conn)
departments = pd.read_sql("SELECT * FROM Departments;", conn)


classroom_courses = pd.merge(classrooms, courses, on='CourseID', how='inner')
class_courses_dep = pd.merge(classroom_courses,departments, on='DepartmentID', how='inner')
class_courses_dep_stud = pd.merge(class_courses_dep,students, on='StudentID', how='inner')

#Question 1

students_departments = class_courses_dep.groupby(["DepartmentID","DepartmentName"]).agg({"StudentID": ['nunique']}).rename(columns = {"StudentID":"Student Number"})
print(students_departments)

#Question 2
students_english_dep = class_courses_dep.query("DepartmentID==1")
students_Eng_Courses = students_english_dep.groupby(["CourseID","CourseName"]).agg({"StudentID": ['count']}).rename(columns = {"StudentID":"Student Number"})
print(students_Eng_Courses)

total_stud_eng_dep = students_english_dep.drop_duplicates(subset='StudentID', keep='first')['StudentID'].count()
print("\nTotal number of student in English department is", total_stud_eng_dep)

#Question 3
students_english_dep = class_courses_dep.query("DepartmentID==2")
students_num_Sci_Courses = students_english_dep.groupby(["CourseID","CourseName"]).agg({"StudentID": ['count']})
students_num_Sci_Courses.loc[:,'Type'] = "Big"
students_num_Sci_Courses.loc[students_num_Sci_Courses.iloc[:,0]>21,'Type'] = "Small"
big_small= students_num_Sci_Courses.groupby(["Type"]).agg({"Type": ['count']})
print(big_small)

#Question 4
print(students.groupby(["Gender"]).agg({"StudentID": ['count']}))
print("The feminist are WRONG, There are more women in the college")

#Question 5
Male_stud = class_courses_dep_stud[class_courses_dep_stud['Gender'] == 'M']
Male_stud = Male_stud.groupby(["CourseName"]).agg({"StudentID": ['count']})

Female_Stud = class_courses_dep_stud[class_courses_dep_stud['Gender'] =='F']
Female_stud = Female_Stud.groupby(["CourseName"]).agg(female=("StudentID" ,"count"))

Male_female= pd.merge(Male_stud,Female_Stud ,on="CourseName")

print(Female_stud)



#gender_course_count = class_courses_dep_stud.groupby(["CourseName","Gender"]).agg({"StudentID": ['count'],'CourseID': ['sum']})
#course_count = class_courses_dep.groupby(["CourseName",]).agg({"StudentID": ['count']})
#print(gender_course_count)
#print(course_count)

KeyError: 'CourseID'